In [1]:
# Read Excel files

import pandas as pd

#  To change
folder_path = "DataSets/example-set/"

df_clients = pd.read_csv(folder_path + 'input_clients.csv')
df_instruments = pd.read_csv(folder_path + 'input_instruments.csv')
df_orders = pd.read_csv(folder_path + 'input_orders.csv')

print(df_clients.to_string())
print(df_instruments.to_string())
print(df_instruments.to_string())

# To read ColumnName
# print(df['ColumnName'].to_string())



  ClientID   Currencies PositionCheck  Rating
0        A      USD,SGD             Y       1
1        B  USD,SGD,JPY             N       2
2        C          SGD             Y       3
3        D          USD             Y       4
4        E          SGD             N       5
  InstrumentID Currency  LotSize
0          SIA      SGD      100
  InstrumentID Currency  LotSize
0          SIA      SGD      100
